# 第 4 章: Apache Spark - 1
このノートブックでは **第 4 章: Apache Spark** における、*Spark でデータ処理を実行する* セクションにおけるスクリプトを実行できます。

## Spark でデータ処理を実行する
### 初期設定

In [ ]:
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"
S3_ENDPOINT = "http://minio:9000"

### 1. SparkSession オブジェクトを初期化する

In [ ]:
from pyspark.sql import SparkSession


spark = (
    SparkSession.builder
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.4,org.apache.hadoop:hadoop-client:3.2.4")
        .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
        .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
        .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT)
        .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
    .getOrCreate()
)

In [ ]:
%sql spark

#### (Optional) データベースの作成
データベースを作成していない場合、以下のセルを実行してください。既にデータベースが存在する場合は、本ステップにつきましてはスキップしてください。

In [ ]:
%%sql
CREATE DATABASE IF NOT EXISTS db

### テーブルを作成する

In [ ]:
%%sql
CREATE TABLE db.sales_data (
    product_name string,
    price decimal(10, 2),
    customer_id bigint,
    order_id string,
    datetime timestamp,
    category string
) USING JSON
LOCATION 's3a://amzn-s3-demo-bucket/spark/data'

In [ ]:
%%sql
CREATE TABLE db.sales_analysis (
  category string,
  total_sales decimal(20,2),
  count_by_year int,
  year int
) USING parquet
LOCATION 's3a://amzn-s3-demo-bucket/spark/analysis'

In [ ]:
%%sql
INSERT INTO db.sales_analysis
SELECT
  category,
  sum(price) AS total_sales,
  count(*) AS count_by_year,
  year(datetime) AS year
FROM db.sales_data
GROUP BY category, year(datetime)

In [ ]:
%%sql
SELECT * FROM db.sales_analysis ORDER BY year DESC, category ASC;

#### Spark DataFrame で集計処理を行う方法

In [ ]:
# Using Spark DataFrame
from pyspark.sql.functions import sum, count, round, year

df.groupBy("category", year("datetime").alias("year")) \
    .agg(round(sum("price")).alias("total_sales"), 
                  count("*").alias("count_by_year")) \
    .select("category", "total_sales", "count_by_year", "year").orderBy('year', ascending=False).show()